In [1]:
import re
import math

WORDS = "../words1.txt"
CROSSWORD = "../crossword1.txt"

In [2]:
def read_words(path):
    f = open(path, "r")
    lines = f.readlines()
    return [line.strip() for line in lines]

words = read_words(WORDS)
print(words)

['to', 'at', 'tea', 'eat']


In [3]:
hashes_regex = re.compile("\.+")

class Line:
    def __init__(self, origin, end):
        self.origin = origin
        self.end = end
    def __repr__(self):
        return "<origin: {}, end: {}>".format(str(self.origin), str(self.end))
    def length(self):
        return max(math.fabs(self.origin[0] - self.end[0]), math.fabs(self.origin[1] - self.end[1]))+1
    def get_type(self):
        if self.is_vertical():
            return "vertical"
        if self.is_horizontal():
            return "horizontal"
    def is_vertical(self):
        return self.origin[0]==self.end[0] and self.origin[1]!=self.origin[1]
    def is_horizontal(self):
        return self.origin[1]==self.end[1] and self.origin[0]!=self.origin[0]
    

def parse_horizontally(path):
    f = open(path, "r")
    horizontal_dots = []
    lines = f.readlines()
    for i, line in enumerate(lines):
        line = line.strip()
        positions = hashes_regex.finditer(line)
        for pos in positions:
            dot_line = Line((pos.span()[0], i), (pos.span()[1]-1, i))
            if dot_line.length()>1:
                horizontal_dots.append(dot_line)
    return horizontal_dots

def get_columns(path):
    f = open(path, "r")
    lines = f.readlines()
    columns = ['' for i in range(len(lines))]
    for line in lines:
        for caract in enumerate(line):
            if caract[1] != "\n":
                columns[caract[0]] += caract[1]
    return columns
    
# NB : the y axis is reversed
def parse_vertically(path):
    vertical_dots = []
    columns = get_columns(path)
    for i, column in enumerate(columns):
        positions = hashes_regex.finditer(column)
        for pos in positions:
            dot_column = Line((i, pos.span()[0]), (i, pos.span()[1]-1))
            if dot_column.length()>1:
                vertical_dots.append(dot_column)
    return vertical_dots

def read_crossword(path):
    # Horizontal lines
    horizontal_lines = parse_horizontally(path)
    # Vertical lines
    vertical_lines = parse_vertically(path)
    return horizontal_lines, vertical_lines
    
hlines, vlines = read_crossword(CROSSWORD)
print(hlines)
print(vlines)

[<origin: (1, 2), end: (3, 2)>]
[<origin: (2, 1), end: (2, 3)>]


In [4]:
def find_intersection(h_line, v_line):
    # Better: use the fact that it is necessary for the intersection to have v_line x position and h_line y position
    a = h_line.origin[0]<= v_line.origin[0]
    b = v_line.origin[0]<=h_line.end[0]
    c = v_line.origin[1]<=h_line.origin[1]
    d = v_line.origin[1]<= h_line.origin[1]
    e = h_line.origin[1]<=v_line.end[1]
    f = v_line.origin[0]>=h_line.origin[0]
    if a and b and c and d and e and f:
        return (v_line.origin[0], h_line.origin[1])
    else:
        return None

def find_intersections(hlines, vlines):
    intersections = []
    """
    [
        {
            "which": (
                nieme ligne de dots hlines,
                mieme colonne de dots vlines
                ),
            "position": (x, y)
    ]
    """
    for i, h_line in enumerate(hlines):
        for j, v_line in enumerate(vlines):
            intersection = find_intersection(h_line, v_line)
            u = intersection[0] - h_line.origin[0]
            v = intersection[1] - v_line.origin[1]
            if intersection != None:
                intersections.append({"position_in_crossword":intersection, "position_in_lines":(u, v), "which":(i, j)})
    return intersections

intersections = find_intersections(hlines, vlines)
print(intersections)

[{'position_in_crossword': (2, 2), 'position_in_lines': (1, 1), 'which': (0, 0)}]


In [5]:
class Crossword:
    def __init__(self, path):
        self.path = path
        self.find_lines()
        self.find_intersections()
    def find_lines(self):
        self.hlines, self.vlines = read_crossword(self.path)
    def find_intersections(self):
        self.intersections = find_intersections(self.hlines, self.vlines)
    def __repr__(self):
        return "horizontal lines: {}\nvertical lines: {}\nintersections: {}>".format(self.hlines, self.vlines, self.intersections)
        
cw = Crossword(CROSSWORD)
print(cw)

horizontal lines: [<origin: (1, 2), end: (3, 2)>]
vertical lines: [<origin: (2, 1), end: (2, 3)>]
intersections: [{'position_in_crossword': (2, 2), 'position_in_lines': (1, 1), 'which': (0, 0)}]>


In [1]:
from constraint_programming import constraint_programming

In [2]:
class CrosswordProblem:
    def __init__(self, wordspath, crosswordspath):
        self.words = read_words(wordspath)
        self.crossword = Crossword(crosswordspath)
    def __repr__(self):
        return "Words: {}\nCrossword:\n{}".format(self.words, self.crossword)

pb = CrosswordProblem(WORDS, CROSSWORD)
pb

NameError: name 'WORDS' is not defined

In [3]:
# A line and its word have the same length
var = {line: set([word for word in words if len(word)==line.length()]) for line in cw.hlines+cw.vlines}
print(var)
P = constraint_programming(var)


NameError: name 'cw' is not defined

In [26]:
lines = cw.hlines + cw.vlines

# A word is used only once
NEQ = {(i, j) for i in words for j in words if i!=j}
for line in lines:
    for other_line in lines:
        if line!=other_line:
            P.addConstraint(line, other_line, NEQ)
print(NEQ)
# Intersections
def get_intersection_possibilities(i, j):
    return {(word, other_word) for word in words for other_word in words if (word[i]==other_word[j] and word!=other_word)}
print(get_intersection_possibilities(0,1))
print(intersections)
for intersection in intersections:
    which_h = intersection['which'][0]
    hline = hlines[which_h]
    which_v = intersection['which'][1]
    vline = vlines[which_v]
    SAME_LETTER_INTERSECT = get_intersection_possibilities(*intersection['position_in_lines'])
    P.addConstraint(hline, vline, SAME_LETTER_INTERSECT)


<origin: (1, 2), end: (3, 2)>
{<origin: (1, 2), end: (3, 2)>: [(<origin: (2, 1), end: (2, 3)>, {('tea', 'at'), ('at', 'to'), ('eat', 'tea'), ('eat', 'at'), ('at', 'eat'), ('to', 'eat'), ('eat', 'to'), ('to', 'at'), ('at', 'tea'), ('to', 'tea'), ('tea', 'to'), ('tea', 'eat')}), (<origin: (2, 1), end: (2, 3)>, {('tea', 'at'), ('at', 'to'), ('eat', 'tea'), ('eat', 'at'), ('at', 'eat'), ('eat', 'to'), ('to', 'eat'), ('to', 'at'), ('at', 'tea'), ('tea', 'to'), ('to', 'tea'), ('tea', 'eat')}), (<origin: (2, 1), end: (2, 3)>, {('tea', 'at'), ('at', 'to'), ('eat', 'tea'), ('eat', 'at'), ('at', 'eat'), ('to', 'eat'), ('eat', 'to'), ('to', 'at'), ('at', 'tea'), ('to', 'tea'), ('tea', 'to'), ('tea', 'eat')}), (<origin: (2, 1), end: (2, 3)>, {('tea', 'at'), ('at', 'to'), ('eat', 'tea'), ('eat', 'at'), ('at', 'eat'), ('eat', 'to'), ('to', 'eat'), ('to', 'at'), ('at', 'tea'), ('tea', 'to'), ('to', 'tea'), ('tea', 'eat')}), (<origin: (2, 1), end: (2, 3)>, {('tea', 'at'), ('at', 'to'), ('eat', 'tea'),

KeyError: <origin: (1, 2), end: (3, 2)>

In [10]:
vline = Line((0,1), (1,0))
vline2 = Line((0,1), (1,0))

In [11]:
a = dict()
a[vline] = 1
a[5] = 'b'
a

{<origin: (0, 1), end: (1, 0)>: 1, 5: 'b'}

In [21]:
a = ['a', 'b']
b = set(a)
conflict = {'toto':[('lala', b)]}
conflict

{'toto': [('lala', {'a', 'b'})]}

In [22]:
c = set(['a', 'a', 'b', 'c'])
print(c)
conflict['toto'].append(('lulu', c))
conflict

{'a', 'c', 'b'}


{'toto': [('lala', {'a', 'b'}), ('lulu', {'a', 'b', 'c'})]}